In [1]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

# Data Preprocessing for Matrix User x Item

In [2]:
#Source Code for data Processing
#https://gist.github.com/victorkohler/f48ea6512058719ba52053851fedc745

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
#np.shape(train_data)
np.shape(test_data)

(3782335, 12)

In [4]:
#train = train_data.append(test_data)
#np.shape(train)

In [5]:
item_metadata = pd.read_csv('item_metadata.csv', sep=',', engine='python')

In [6]:
train_v2 = train_data[train_data['action_type']=='clickout item']
train_v2= train_v2[train_v2.reference.notnull()]
train_v2.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...
115,02SRUT1NQYH1,3599a6f709eab,1541063864,35,clickout item,2795374,FI,"Krakow, Poland",mobile,NaN,2795374|5582964|1088390|2781070|1258068|127196...,64|54|36|121|76|81|92|40|73|52|98|104|56|414|6...
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...
122,03K8AXBL4BX2,ec139e10b9238,1541100652,7,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...


In [7]:
train_v2.dtypes

user_id            object
session_id         object
timestamp           int64
step                int64
action_type        object
reference          object
platform           object
city               object
device             object
current_filters    object
impressions        object
prices             object
dtype: object

In [8]:
train_v2["reference"]= train_v2["reference"].astype(int)
train_v2["step"]= train_v2["step"].astype(int)
train_v2.dtypes

user_id            object
session_id         object
timestamp           int64
step                int32
action_type        object
reference           int32
platform           object
city               object
device             object
current_filters    object
impressions        object
prices             object
dtype: object

In [9]:
train_v2.head()
#np.shape(train_v2)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...
115,02SRUT1NQYH1,3599a6f709eab,1541063864,35,clickout item,2795374,FI,"Krakow, Poland",mobile,NaN,2795374|5582964|1088390|2781070|1258068|127196...,64|54|36|121|76|81|92|40|73|52|98|104|56|414|6...
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...
122,03K8AXBL4BX2,ec139e10b9238,1541100652,7,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...


In [10]:
#train_v3 = train_v2.head(10000)
train_v3 = train_v2
train_v3.head()
np.shape(train_v3)

(1586586, 12)

In [11]:
data = train_v3[['user_id','reference']]
data.head()
#np.shape(data)

,user_id,reference
13,00RL8Z82B2Z1,109038
15,00RL8Z82B2Z1,1257342
115,02SRUT1NQYH1,2795374
121,03K8AXBL4BX2,1032816
122,03K8AXBL4BX2,1032816


In [12]:
data=data.groupby(['user_id','reference']).size().reset_index()
data.columns = ['user', 'item', 'click_count']
data.head()

,user,item,click_count
0,0001VQMGUI65,477811,1
1,0001VQMGUI65,950829,1
2,0001VQMGUI65,2019467,1
3,0001VQMGUI65,3133074,2
4,0003QTCX5MJX,2195060,1


In [13]:
user_id_table = data['user']
user_id_table = pd.DataFrame(user_id_table)
user_id_table = pd.DataFrame(user_id_table['user'].unique())
user_id_table.columns = ['user']
user_id_table['user_id'] = dict(enumerate(user_id_table['user'], start=0))
user_id_table.head()
#np.shape(user_id_table)

,user,user_id
0,0001VQMGUI65,0
1,0003QTCX5MJX,1
2,0004IOZI7CKF,2
3,0004WCFRV3FB,3
4,0006W0R5A5V8,4


In [14]:
item_id_table = data['item']
item_id_table = pd.DataFrame(item_id_table)
item_id_table = pd.DataFrame(item_id_table['item'].unique())
item_id_table.columns = ['item']
item_id_table['item_id'] = dict(enumerate(item_id_table['item'], start=0))
item_id_table.head()
#np.shape(item_id_table)

,item,item_id
0,477811,0
1,950829,1
2,2019467,2
3,3133074,3
4,2195060,4


In [15]:
data = data.merge(user_id_table, how='left', left_on='user', right_on = 'user').merge(item_id_table, how='left', left_on='item', right_on = 'item')
data.head()
#data = data[["user_id", "item_id","clickcount"]]
#data.head()

,user,item,click_count,user_id,item_id
0,0001VQMGUI65,477811,1,0,0
1,0001VQMGUI65,950829,1,0,1
2,0001VQMGUI65,2019467,1,0,2
3,0001VQMGUI65,3133074,2,0,3
4,0003QTCX5MJX,2195060,1,1,4


In [16]:
data2 = data[["user_id", "item_id","click_count"]]
data2.head()

,user_id,item_id,click_count
0,0,0,1
1,0,1,1
2,0,2,1
3,0,3,2
4,1,4,1


In [18]:
#Based on the implementation from https://github.com/ivallesp/deep_matrix_factorization
def batching(list_of_iterables, n=1, infinite=False, return_incomplete_batches=False):
    list_of_iterables = [list_of_iterables] if type(list_of_iterables) is not list else list_of_iterables
    assert(len({len(it) for it in list_of_iterables}) == 1)
    l = len(list_of_iterables[0])
    while 1:
        for ndx in range(0, l, n):
            if not return_incomplete_batches:
                if (ndx+n) > l:
                    break
            yield [iterable[ndx:min(ndx + n, l)] for iterable in list_of_iterables]

        if not infinite:
            break

In [19]:
#Based on the implementation from https://github.com/ivallesp/deep_matrix_factorization
import tensorflow as tf
import numpy as np

class DeepFactorization():
    def __init__(self, n_users, n_items, emb_size, lr=0.001, _lambda = 0.1):
        self.lr = lr
        self._lambda = _lambda
        tf.reset_default_graph()
        self.ph_keep_prob = tf.placeholder(dtype=tf.float32, shape=(None), name="keep_prob")
        self.ph_u_ids=tf.placeholder(dtype=tf.int32, shape=(None,), name="u_ids_ph")
        self.ph_i_ids=tf.placeholder(dtype=tf.int32, shape=(None,), name="u_ids_ph")
        self.ph_y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name="y")
        self.u_emb = tf.get_variable(shape=(n_users, emb_size), name="U_Embedding")
        self.i_emb = tf.get_variable(shape=(n_items, emb_size), name="I_Embedding")
        self.u_bias = tf.get_variable(shape=(n_users, 1), name="U_Bias")
        self.i_bias = tf.get_variable(shape=(n_items, 1), name="I_Bias")
        u=tf.nn.embedding_lookup(self.u_emb, self.ph_u_ids) #+ tf.nn.embedding_lookup(self.u_bias, self.ph_u_ids) 
        i=tf.nn.embedding_lookup(self.i_emb, self.ph_i_ids) #+ tf.nn.embedding_lookup(self.i_bias, self.ph_i_ids)
        ub=tf.nn.embedding_lookup(self.u_bias, self.ph_u_ids)
        ib=tf.nn.embedding_lookup(self.i_bias, self.ph_i_ids)
        ub=tf.nn.dropout(ub, self.ph_keep_prob)
        ib=tf.nn.dropout(ib, self.ph_keep_prob)
        u=tf.nn.dropout(u, self.ph_keep_prob)
        i=tf.nn.dropout(i, self.ph_keep_prob)
        self.mfac = tf.reduce_sum(u*i + ub + ib, axis=1, keepdims=True)
        self.mfac = tf.nn.dropout(self.mfac, self.ph_keep_prob)
        emb = tf.concat([u, i], axis=1)
        emb = tf.nn.dropout(emb, self.ph_keep_prob)
        h = tf.layers.Dense(128, tf.nn.swish)(emb)
        h = tf.nn.dropout(h, self.ph_keep_prob)
        h = tf.layers.Dense(64, tf.nn.swish)(h)
        h = tf.nn.dropout(h, self.ph_keep_prob)
        h = tf.layers.Dense(32, tf.nn.swish)(h)
        h = tf.nn.dropout(h, self.ph_keep_prob)
        self.output = tf.layers.Dense(1)(h) + self.mfac
        self.loss = tf.losses.mean_squared_error(self.ph_y, self.output)
        self.op = tf.train.RMSPropOptimizer(learning_rate=lr).minimize(self.loss)
        self.sess=tf.Session()
        self.sess.run(tf.global_variables_initializer())
        
    def fit(self, x, y, batch_size):
        batcher = batching([list(x), list(y)], n=batch_size, return_incomplete_batches=True)
        for batch_x, batch_y in batcher:
            self.train_on_batch(batch_x, batch_y)
        
    def train_on_batch(self, x, y):
        u_ids, i_ids = list(zip(*list(x)))                              
        self.sess.run(self.op, feed_dict={self.ph_u_ids: u_ids, self.ph_i_ids: i_ids, self.ph_y: y,
                                          self.ph_keep_prob: 1-self._lambda})
        
    
    def predict(self, x, batch_size):
        batcher = batching([list(x)], n=batch_size, return_incomplete_batches=True)
        preds = []
        for batch_x in batcher:
            batch_x = batch_x[0]
            u_ids, i_ids = list(zip(*list(batch_x)))
            preds.append(self.sess.run(self.output, feed_dict={self.ph_u_ids: u_ids, self.ph_i_ids: i_ids,
                                                      self.ph_keep_prob: 1.0}))
        preds = np.row_stack(preds)
        return preds
    
    def evaluate(self, x, y, batch_size):
        return np.mean((np.round(self.predict(x, batch_size)) - y)**2)

In [29]:
data.head()

,user,item,click_count,user_id,item_id
0,0001VQMGUI65,477811,1,0,0
1,0001VQMGUI65,950829,1,0,1
2,0001VQMGUI65,2019467,1,0,2
3,0001VQMGUI65,3133074,2,0,3
4,0003QTCX5MJX,2195060,1,1,4


In [ ]:
# Load the data and calculate the users set and items set cardinalities
import itertools

original_itemid = data["item"].tolist()

df = data2
#df = df.iloc[:5000,:5000] #slicing for testing smaller sample
U_cardinality,I_cardinality = np.unique(df['user_id']).shape[0], np.unique(df['item_id']).shape[0]
mat = df.values
train_mat = mat
x_train=(train_mat[:,:2])
y_train=(train_mat[:,2:])

DF = DeepFactorization(n_users=U_cardinality, n_items=I_cardinality, emb_size=3, lr=0.002, _lambda=0.1)
losses_train = [DF.evaluate(x_train, y_train, batch_size=200)] # Add the initial loss (w. random weights)
for i in range(50): # Run for 50 epochs
    DF.fit(x_train, y_train, batch_size=128)
    losses_train.append(DF.evaluate(x_train, y_train, batch_size=128)) # Compute the train performance

y = list(range(50))
items = dict()
for i in range(1): #make recomendations for each user
    print("\nuser",i,"recommendations")
    print()
    x = (i,)
    user_item_iterable = list(itertools.product(x, y))
    predictions = DF.predict(user_item_iterable,200)
    #print(predictions)
    for j in range(len(predictions)):
        items[j] = (original_itemid[j],predictions[j][0])
    recommendations = pd.DataFrame.from_dict(items, orient='index',columns=["item","score"])
    print(recommendations)